### Nama : Rizky Ramdhani Koswara
### NPM : 11122300
### Kelas : 4KA25

In [2]:
# 1. Impor Library yang Dibutuhkan
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Reshape, Flatten, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
import glob

# Membuat direktori untuk menyimpan gambar jika belum ada
if not os.path.exists('generated_images'):
    os.makedirs('generated_images')

# 2. Definisi Variabel Global
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100
# Menggunakan optimizer Adam dengan parameter modern
optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# 3. Membangun Model Generator
def build_generator():
    model = Sequential(name="Generator")
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    model.summary()
    return model

# 4. Membangun Model Diskriminator
def build_discriminator():
    model = Sequential(name="Discriminator")
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2)) # Menambahkan layer yang hilang
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

# Membuat instance generator dan diskriminator
generator = build_generator()
discriminator = build_discriminator()

# Meng-compile diskriminator
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# 5. Menggabungkan Model untuk Membentuk GAN
# Saat melatih generator, bobot diskriminator tidak diubah (dibekukan)
discriminator.trainable = False

GAN = Sequential(name="GAN")
GAN.add(generator)
GAN.add(discriminator)
GAN.compile(loss='binary_crossentropy', optimizer=optimizer)
GAN.summary()

# 6. Fungsi untuk Menyimpan Gambar
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    # Menskalakan ulang gambar dari [-1, 1] ke [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    # Menyimpan gambar dengan nama file berdasarkan epoch
    fig.savefig(f"generated_images/{epoch}.png")
    plt.close()
    print(f"Image saved for epoch {epoch}")

# 7. Fungsi untuk Melatih GAN
def train(epochs, batch_size=128, save_interval=500):
    (X_train, _), (_, _) = mnist.load_data()
    # Menskalakan data ke rentang [-1, 1]
    X_train = (X_train.astype(np.float32) - 127.5) / 127.5
    # Menambahkan dimensi channel
    X_train = np.expand_dims(X_train, axis=3)
    # Label target
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # --- Melatih Diskriminator ---
        imgs = X_train[np.random.randint(0, X_train.shape[0], batch_size)]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        # --- Melatih Generator ---
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = GAN.train_on_batch(noise, valid)
        # Mencetak progres
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {d_loss[1]*100:.2f}%] [G loss: {g_loss:.4f}]")
        # Menyimpan gambar
        if epoch % save_interval == 0:
            save_imgs(epoch)

# Mulai pelatihan (disarankan epoch lebih rendah untuk percobaan awal)
train(epochs=10001, batch_size=128, save_interval=1000)

# 8. Membuat GIF dari Hasil Pelatihan
anim_file = 'dcgan_mnist.gif'
with imageio.get_writer(anim_file, mode='I') as writer:
    filenames = glob.glob('generated_images/*.png')
    filenames = sorted(filenames, key=lambda x: int(os.path.basename(x).split('.')[0]))
    if filenames:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
        # Ulangi frame terakhir beberapa kali
        for _ in range(5):
            writer.append_data(image)
        print(f"GIF created: {anim_file}")
    else:
        print("No images found to create a GIF.")

Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                 │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 784)            │       803,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,493,520 (5.70 MB)

 Trainable params: 1,489,936 (5.68 MB)

 Non-trainable params: 3,584 (14.00 KB)

...

 Total params: 2,027,025 (7.73 MB)

 Trainable params: 1,489,936 (5.68 MB)

 Non-trainable params: 537,089 (2.05 MB)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step 
0 [D loss: 0.8698, acc.: 35.94%] [G loss: 0.9071]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 189ms/step
Image saved for epoch 0
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1 [D loss: 0.8139, acc.: 47.14%] [G loss: 0.8750]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2 [D loss: 0.8023, acc.: 47.16%] [G loss: 0.8544]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
3 [D loss: 0.8051, acc.: 46.96%] [G loss: 0.8241]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4 [D loss: 0.8094, acc.: 45.01%] [G loss: 0.7994]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
5 [D loss: 0.8132, acc.: 42.44%] [G loss: 0.7726]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
6 [D loss: 0.8191, acc.: 39.52%] [G loss: 0.7493]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
7 [D loss: 0.8247, acc.: 37.39%] [G loss: 0.7265]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
8 [D loss: 0.8325, acc.: 35.31%] [G loss: 0.7064]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
9 [D loss: 0.8432, acc.: 32.99%] [G loss: 0.6852]
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
10 [D loss: 0.8561

KeyboardInterrupt: 

Kode ini membangun sebuah Generative Adversarial Network (GAN), yaitu sebuah model deep learning yang mampu menghasilkan data baru yang orisinal. Dalam kasus ini, tujuannya adalah untuk menghasilkan gambar tulisan tangan angka (0-9) yang terlihat realistis, seolah-olah berasal dari dataset MNIST.

Logika utamanya adalah "persaingan" antara dua jaringan saraf: Generator dan Diskriminator.

## 1. Inisialisasi dan Konfigurasi Awal
Blok pertama kode berfungsi untuk menyiapkan semua "bahan baku" yang diperlukan.

Impor Library: Kode mengimpor semua alat yang dibutuhkan:

keras / tensorflow.keras: Kerangka kerja utama untuk membangun model jaringan saraf.

numpy: Untuk operasi matematika pada data (gambar direpresentasikan sebagai array angka).

matplotlib: Untuk membuat dan menyimpan visualisasi gambar yang dihasilkan.

os, glob, imageio: Utilitas untuk mengelola file dan membuat file GIF dari hasil gambar.

Konfigurasi Parameter: Variabel-variabel penting didefinisikan di sini.

img_shape = (28, 28, 1): Menentukan bahwa gambar yang akan kita buat berukuran 28x28 piksel dengan 1 channel warna (hitam-putih atau grayscale).

latent_dim = 100: Ini adalah ukuran ruang laten. Bayangkan ini sebagai 100 "tombol" atau "slider" acak yang akan digunakan oleh Generator sebagai inspirasi untuk melukis sebuah gambar. Setiap kombinasi dari 100 angka acak ini akan menghasilkan gambar yang unik.

optimizer = Adam(...): Memilih Adam sebagai algoritma optimisasi. Tugasnya adalah secara cerdas menyesuaikan "bobot" atau parameter di dalam Generator dan Diskriminator selama pelatihan agar performa mereka meningkat.

## 2. Arsitektur Generator (Si Seniman Pemalsu )
Tujuan: Menghasilkan gambar palsu yang terlihat asli.

Logika kerjanya adalah mengubah input acak yang kecil menjadi sebuah gambar yang kompleks.

Input: Menerima input berupa vektor acak berukuran latent_dim (100 angka).

Proses "Upscaling": Melalui serangkaian lapisan Dense, Generator secara bertahap memperbesar dimensi data dari 100 menjadi 256, lalu 512, 1024, dan akhirnya menjadi 784 neuron (28 * 28). Ini seperti mengubah sketsa kasar menjadi lukisan yang lebih detail.

LeakyReLU & BatchNormalization: Digunakan setelah setiap lapisan Dense untuk menstabilkan proses pelatihan dan membantu model belajar lebih cepat dan efektif.

Output: Lapisan terakhir menggunakan fungsi aktivasi tanh, yang memastikan nilai setiap piksel berada di rentang [-1, 1].

Reshape: Akhirnya, vektor 784 piksel tersebut diubah bentuknya menjadi format gambar (28, 28, 1).

Analogi: Generator adalah seorang seniman pemalsu. Ia tidak pernah melihat lukisan asli, tapi ia terus mencoba melukis sesuatu berdasarkan deskripsi acak (latent_dim), dan berharap lukisannya bisa menipu seorang kritikus seni.

## 3. Arsitektur Diskriminator (Si Kritikus Seni )
Tujuan: Membedakan antara gambar asli (dari dataset MNIST) dan gambar palsu (dari Generator).

Logika kerjanya adalah seperti model klasifikasi gambar pada umumnya.

Input: Menerima sebuah gambar berukuran (28, 28, 1).

Flatten: Gambar 2D diratakan menjadi vektor 1D (784 piksel) agar bisa diproses oleh lapisan Dense.

Proses Klasifikasi: Melalui lapisan Dense (512 dan 256 neuron), Diskriminator mencoba mengenali pola dan fitur dari gambar tersebut.

Output: Lapisan terakhir hanya memiliki 1 neuron dengan aktivasi sigmoid. Fungsinya adalah mengeluarkan satu nilai probabilitas antara 0 dan 1.

Nilai mendekati 1 berarti "Saya yakin ini gambar asli."

Nilai mendekati 0 berarti "Saya yakin ini gambar palsu."

Analogi: Diskriminator adalah seorang kritikus seni. Tugasnya hanya satu: melihat sebuah karya seni dan memutuskan apakah itu karya asli atau palsu.

## 4. Model Gabungan (Arena Pertarungan )
Di sinilah kedua jaringan digabungkan untuk dilatih bersama.

Pembekuan Diskriminator: Langkah paling krusial adalah discriminator.trainable = False. Ini berarti saat kita melatih model gabungan ini, hanya bobot Generator yang akan diperbarui.

Penyusunan: Model GAN disusun dengan generator terlebih dahulu, lalu discriminator. Alurnya:

Input acak (latent_dim) -> Generator -> Gambar Palsu -> Diskriminator -> Output (0 atau 1)

Tujuan Pelatihan Model GAN: Tujuan dari model gabungan ini adalah untuk melatih Generator. Kita memberikan gambar palsu ke Diskriminator dan "berbohong" dengan mengatakan bahwa itu adalah gambar asli (memberi label 1). Kerugian (loss) yang dihasilkan dari kebohongan ini digunakan sebagai sinyal untuk memperbaiki Generator.

Logikanya: Kita melatih si seniman pemalsu (Generator) dengan cara melihat seberapa baik ia bisa menipu si kritikus seni (Diskriminator) yang kemampuannya sedang "dibekukan" sementara.

## 5. Proses Pelatihan (Training Loop)
Ini adalah inti dari proses pembelajaran GAN, yang dilakukan secara bergantian.

Bayangkan setiap epoch adalah satu ronde pertarungan:

Ronde 1: Melatih si Kritikus (Diskriminator)

Ambil beberapa gambar asli dari dataset MNIST. Latih Diskriminator untuk mengenali ini sebagai "asli" (label 1).

Minta Generator untuk membuat beberapa gambar palsu. Latih Diskriminator untuk mengenali ini sebagai "palsu" (label 0).

Tujuan langkah ini adalah membuat Diskriminator semakin pintar dalam membedakan yang asli dan yang palsu.

Ronde 2: Melatih si Pemalsu (Generator)

Minta Generator untuk membuat satu set gambar palsu baru.

Masukkan gambar-gambar palsu ini ke dalam model GAN gabungan (di mana Diskriminator dibekukan).

Tipu modelnya! Beri tahu model bahwa gambar-gambar palsu ini adalah "asli" (label 1).

Model akan menghitung seberapa parah Diskriminator "tertipu". Error atau loss dari proses ini hanya digunakan untuk memperbarui bobot Generator.

Tujuan langkah ini adalah membuat Generator semakin pintar dalam menipu Diskriminator.

Proses dua ronde ini diulang ribuan kali. Seiring waktu, Generator akan menghasilkan gambar yang semakin realistis, dan Diskriminator akan semakin ahli dalam membedakannya, menciptakan sebuah keseimbangan di mana gambar yang dihasilkan menjadi sangat baik.